In [10]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

from snowflake.connector import connect

In [11]:
load_dotenv()

True

In [12]:
username = os.environ.get('SNOWFLAKE_USERNAME')
password = os.environ.get('SNOWFLAKE_PASSWORD')
account = os.environ.get('SNOWFLAKE_ACCOUNT')

In [13]:
con = connect(
    user=username,
    password=password,
    account=account
)

In [14]:
con

In [15]:
cursor = con.cursor()

In [16]:
def run_sql(cursor, query, single_row=False, print_results=False):
    exe = cursor.execute(query)
    print(f"Execution: {exe}")
    if single_row is True:
        result = cursor.fetchone()
        if print_results is True: print(f"Result: {result}")
        return result
    results = cursor.fetchall()
    if print_results is True: print(f"Results: {results}")
    return results
    

In [17]:
run_sql(cursor, "USE WAREHOUSE COMPUTE_WH")

Execution: <snowflake.connector.cursor.SnowflakeCursor object at 0x7f9dfbff78b0>


[('Statement executed successfully.',)]

In [18]:
run_sql(cursor,'USE DATABASE AIRBNB_CALENDAR_LISTINGS_SAMPLE')

Execution: <snowflake.connector.cursor.SnowflakeCursor object at 0x7f9dfbff78b0>


[('Statement executed successfully.',)]

In [19]:
practice_query = """
                SELECT DISTINCT 
                    c."listing_id", c."price", p."reviews_count", p."city", 
                    p."room_type_category", p."space_type", p."bedrooms", p."bathrooms"
                FROM "AIRBNB_CALENDAR_SAMPLE" c
                JOIN "AIRBNB_LISTINGS_SAMPLE" p ON p."listing_id" = c."listing_id" AND p."crawled_at_ds" = c."ds"
                WHERE 
                    c."price" IS NOT NULL
                        AND
                    c."currency" = 'USD'
                ORDER BY c."price" DESC
                --LIMIT 1000000
                """

In [20]:
data = run_sql(cursor, practice_query)

Execution: <snowflake.connector.cursor.SnowflakeCursor object at 0x7f9dfbff78b0>


In [21]:
len(data)

2589842

In [22]:
column_names = ['listing_id', 'price', 'review_counts',\
                'city', 'room_type_category', 'space_type', 'bedrooms', 'bathrooms']

In [23]:
df_airbnb = pd.DataFrame(data, columns=column_names)

In [24]:
df_airbnb.head()

,listing_id,price,review_counts,city,room_type_category,space_type,bedrooms,bathrooms
0,33196476,1983148,81,,entire_home,Entire rental unit,4.0,4.0
1,33196476,1272563,81,,entire_home,Entire rental unit,4.0,4.0
2,40900114,60287,0,San Andrés,private_room,Hotel room,2.0,3.0
3,40900114,60053,0,San Andrés,private_room,Hotel room,2.0,3.0
4,40900114,59829,0,San Andrés,private_room,Hotel room,2.0,3.0


In [25]:
col1, col2 = "price", "review_counts"
corr = df_airbnb[col1].corr(df_airbnb[col2])
print(corr)

-0.016635976062880566


In [26]:
col1, col2 = "price", "bedrooms"
corr = df_airbnb[col1].corr(df_airbnb[col2])
print(corr)

0.06512141814911342


In [27]:
col1, col2 = "price", "bathrooms"
corr = df_airbnb[col1].corr(df_airbnb[col2])
print(corr)

0.05741000671447218


In [28]:
WEATHER_QUERIES = {
    'USE_DB': '''
            USE WEATHERSOURCE_TILE_SAMPLE_SNOWFLAKE_SECURE_SHARE_1641488329256;
            ''',
    'GET_POSTAL_CODES': '''    
                        SELECT DISTINCT
                            POSTAL_CODE
                        FROM 
                            standard_tile.forecast_day
                        WHERE
                            COUNTRY = 'US';
                        '''
}

In [29]:
run_sql(cursor, WEATHER_QUERIES['USE_DB'])

Execution: <snowflake.connector.cursor.SnowflakeCursor object at 0x7f9dfbff78b0>


[('Statement executed successfully.',)]

In [30]:
postal_codes = run_sql(cursor, WEATHER_QUERIES['GET_POSTAL_CODES'])

Execution: <snowflake.connector.cursor.SnowflakeCursor object at 0x7f9dfbff78b0>


In [31]:
postal_codes

[('98547',),
 ('98557',),
 ('23086',),
 ('26425',),
 ('82215',),
 ('48130',),
 ('48350',),
 ('03858',),
 ('83246',),
 ('32521',),
 ('62475',),
 ('20045',),
 ('49101',),
 ('06751',),
 ('39426',),
 ('75605',),
 ('35087',),
 ('29639',),
 ('37203',),
 ('93381',),
 ('60141',),
 ('78938',),
 ('60445',),
 ('51248',),
 ('27958',),
 ('24512',),
 ('24572',),
 ('93741',),
 ('75476',),
 ('29734',),
 ('73459',),
 ('95136',),
 ('20427',),
 ('46202',),
 ('55397',),
 ('54555',),
 ('12986',),
 ('77461',),
 ('31042',),
 ('11717',),
 ('57051',),
 ('39571',),
 ('77865',),
 ('78056',),
 ('31097',),
 ('73620',),
 ('66094',),
 ('47542',),
 ('22637',),
 ('77055',),
 ('45873',),
 ('92628',),
 ('11730',),
 ('22313',),
 ('61541',),
 ('18248',),
 ('65066',),
 ('67651',),
 ('75790',),
 ('98548',),
 ('75245',),
 ('93777',),
 ('76369',),
 ('65809',),
 ('80217',),
 ('97540',),
 ('70554',),
 ('31601',),
 ('02461',),
 ('55012',),
 ('29122',),
 ('33401',),
 ('13746',),
 ('77004',),
 ('79698',),
 ('75027',),
 ('39235',),